In [7]:
sys.path.append(os.path.abspath('../..')) # include top level package in python path

In [8]:
import torch
from comparison.examples.vae_mnist import VAE_MNIST
from comparison.loss import ELBO
from tqdm.notebook import tqdm

In [13]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_dataset = datasets.MNIST('./_mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                   ]))

test_dataset = datasets.MNIST('./_mnist', train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                   ]))

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [10]:
def train(model, optim, dataloader, no_epochs=3):
    model.train()
    
    train_loss = []
    for epoch in range(1, no_epochs + 1):
        for imgs, _ in tqdm(dataloader, desc="Training Epoch №%s" % epoch, leave=False):
            imgs = imgs.view(-1, 28*28)
            vae_res = model(imgs)
            loss = ELBO(vae_res)
            train_loss.append(loss.item())

            optim.zero_grad()
            loss.backward()
            optim.step()

    return torch.Tensor(train_loss)

In [14]:
model = VAE_MNIST()
optim = torch.optim.Adam(model.parameters(), lr=1e-4)

In [15]:
train(model, optim, train_loader)

Training Epoch №0:   0%|          | 0/469 [00:00<?, ?it/s]

Training Epoch №1:   0%|          | 0/469 [00:00<?, ?it/s]

Training Epoch №2:   0%|          | 0/469 [00:00<?, ?it/s]

tensor([-7.3797e+02, -7.4655e+02, -7.4888e+02,  ..., -4.8170e+20,
        -2.4211e+19, -2.7775e+18])